<a href="https://colab.research.google.com/github/Yogender-Singh/Notebooks/blob/main/yolov5_custom_data_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Clone YOLOv5 Repository and Install Requirements

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5/
!pip install -r requirements.txt -qq

### Download Dataset

In [ ]:
%cd ..
!mkdir vehicles_open_image
%cd vehicles_open_image
!curl -L -s "https://public.roboflow.com/ds/2Tb6yXY8l8?key=Eg82WpxUEr" > vehicles.zip
!unzip -q vehicles.zip
!rm vehicles.zip

## Blog Post Code

### Import Packages

In [ ]:
import yaml
import os
import glob
from IPython.display import Image, display

### Create Data Configuration

In [ ]:
config = {
    "path": "/content/vehicles_open_image",
    "train": "/content/vehicles_open_image/train",
    "val": "/content/vehicles_open_image/valid",
    "nc": 5,
    "names": ["Ambulance", "Bus", "Car", "Motorcycle", "Truck"]
}

with open("data.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

%cd /content/yolov5

### Define Hyperparameters for the Model

In [ ]:
SIZE = 640
BATCH_SIZE = 32
EPOCHS = 20
MODEL = "yolov5s"
WORKERS = 1
PROJECT = "parking_lot"
RUN_NAME = f"{MODEL}_size{SIZE}_epochs{EPOCHS}_batch{BATCH_SIZE}_small"

### Training with all layers

In [ ]:
!python train.py --img {SIZE}\
                --batch {BATCH_SIZE}\
                --epochs {EPOCHS}\
                --data ../vehicles_open_image/data.yaml\
                --weights {MODEL}.pt\
                --workers {WORKERS}\
                --project {PROJECT}\
                --name {RUN_NAME}\

### Detector Sample Training Data and Performance Metrics

In [ ]:
runs_directory = "parking_lot/yolov5s_size640_epochs20_batch32_small"

display(
    Image(
        filename=f"{runs_directory}/train_batch0.jpg"
    )
)

In [ ]:
display(
    Image(
        filename=f"{runs_directory}/results.png"
    )
)

In [ ]:
display(
    Image(
        filename=f"{runs_directory}/val_batch2_labels.jpg"
    )
)
print("\n")
display(
    Image(
        filename=f"{runs_directory}/val_batch2_pred.jpg"
    )
)

### Run Inference  With Trained Weights

Run inference with a pretrained pytorch checkpoint on contents of `test/images` folder inside the `vehicles_open_image` dataset.

In [ ]:
!python detect.py --weights {runs_directory}/weights/best.pt\
                  --source ../vehicles_open_image/test/images\
                  --conf 0.1\
                  --data ../vehicles_open_image/data.yaml\
                  --project {PROJECT}/{RUN_NAME}\
                  --name test_data_detections

In [ ]:
#display inference on few test images
for imageName in glob.glob(runs_directory+'/test_data_detections/*.jpg')[60:65]:
    display(Image(filename=imageName))
    print("\n")

### Fine-tuning with 10 layers freezed

In [ ]:
!python train.py --img {SIZE}\
                --batch {BATCH_SIZE}\
                --epochs {EPOCHS}\
                --data ../vehicles_open_image/data.yaml\
                --weights {MODEL}.pt\
                --workers {WORKERS}\
                --project {PROJECT}\
                --name freeze_layers\
                --freeze 0 1 2 3 4 5 6 7 8 9 10 

### Performance Metric of Freeze Model

In [ ]:
runs_directory = "parking_lot/freeze_layers"

display(
    Image(
        filename=f"{runs_directory}/train_batch0.jpg"
    )
)

In [ ]:
display(
    Image(
        filename=f"{runs_directory}/results.png"
    )
)

In [ ]:
display(
    Image(
        filename=f"{runs_directory}/val_batch2_labels.jpg"
    )
)
print('\n')
display(
    Image(
        filename=f"{runs_directory}/val_batch2_pred.jpg"
    )
)

### Run Inference  With Trained Weights

Run inference with a pretrained pytorch checkpoint on contents of `test/images` folder inside the `vehicles_open_image` dataset.

In [ ]:
!python detect.py --weights {runs_directory}/weights/best.pt\
                  --source ../vehicles_open_image/test/images\
                  --conf 0.1\
                  --data ../vehicles_open_image/data.yaml\
                  --project {runs_directory}\
                  --name test_data_detections

In [ ]:
#display inference on few test images
for imageName in glob.glob(f"{runs_directory}//test_data_detections/*.jpg")[60:65]: 
    display(Image(filename=imageName))
    print("\n")